#### Selenium Для Python. Парсим динамические и закрытые данные
1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pymongo import MongoClient

DEBUG = True 

In [2]:
LOGIN = ''
PASSWORD = ''

In [3]:
#Авторизация 
def login_to_mail_ru(driver , login=LOGIN, password = PASSWORD):
    driver.get("https://mail.ru/") 
    elem = driver.find_element_by_id('mailbox:login')
    elem.send_keys(login)
    elem = driver.find_element_by_id('mailbox:password')
    elem.send_keys(password)
    elem.send_keys(Keys.RETURN)
    time.sleep(10)
    if "Входящие - Почта Mail.ru" in driver.title:
        return True
    else:
        return False
#Выход их аккаунта    
def logout_mail_ru(driver):
    time.sleep(10)
    driver.find_element_by_id('PH_logoutLink').click()
    print('Осуществлен выход из аккаунта')
    driver.close()
    print('Закрыт браузер')
    
#Парсинг страницы с одним письмом    
def parse_one_email_mail_ru(driver):
    letter_author = driver.find_element_by_class_name('letter__contact-item').get_attribute('title')
    letter_date = driver.find_element_by_class_name('letter__date').text 
    letter_subject = driver.find_element_by_class_name('thread__subject').text 
    letter_text = driver.find_element_by_class_name('letter-body__body-content').text
    return {'letter_author':letter_author, 
            'letter_date':letter_date, 
            'letter_subject':letter_subject, 
            'letter_text':letter_text}

#Парсинг раздела Входящие
def parse_emails_mail_ru(driver):
    emails = []
    len_elements = len(driver.find_elements_by_class_name('llc'))
    print(f'Найдено {len_elements} входящих писем')
    for i in range(len_elements):
        driver.find_elements_by_class_name('llc')[i].click()        
        time.sleep(10)
        emails.append(parse_one_email_mail_ru(driver))
        driver.back()
        time.sleep(10)
    print(f'Обработано {len(emails)} писем.')
    return emails

#Основная функция
def main_parse_emails_mail_ru(login=LOGIN, password = PASSWORD):
    driver = webdriver.Firefox()
    driver.implicitly_wait(10)
    if login_to_mail_ru(driver, login, password):
        print('Авторизация прошла успешно')
        emails = parse_emails_mail_ru(driver)
        if len(emails) != 0:
            save_emails_mail_ru(emails)
        logout_mail_ru(driver)
    else:
        print(f'Авторизация прошла неудачно: {driver.title}')

#Функция сохранения в базу данных
def save_emails_mail_ru(emails):
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client['test']
    mail_ru_collection = db.mail_ru
    
    if DEBUG:
        mail_ru_collection.drop()
        
    result = mail_ru_collection.insert_many(emails)    
    print(f'Сохранено {len(result.inserted_ids)}')

In [4]:
main_parse_emails_mail_ru(LOGIN, PASSWORD)

Авторизация прошла успешно
Найдено 4 входящих писем
Обработано 4 писем.
Сохранено 4
Осуществлен выход из аккаунта
Закрыт браузер



2) Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [5]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
#Основная функция
def parse_hits_mvideo():
    result = []
    driver = webdriver.Firefox()
    driver.implicitly_wait(10)
    driver.get("https://www.mvideo.ru/") 
    time.sleep(10)
    # Избавляемся от баннера
    WebDriverWait(driver,30).until(EC.presence_of_all_elements_located((By.ID, 'fl-226164')))
    action = ActionChains(driver)
    action.click()
    action.perform()
    
    hits_gallery = driver.find_elements_by_class_name('accessories-new')[0]  # Хиты продаж первые на странице  
    next_btn = hits_gallery.find_element_by_class_name('next-btn')
    db_hists_mvideo = init_db()
    for i in range(5):
        hits = hits_gallery.find_elements_by_class_name('gallery-list-item')
        for hit in hits[:4]:
            title = hit.find_element_by_tag_name('h4').get_attribute('title')
            price = hit.find_element_by_class_name('c-pdp-price__current').text
            insert_hits(db_hists_mvideo, {
               'title':title,
               'price':price 
            })
        next_btn.click() 
        time.sleep(10)
        
    driver.close()
    
def init_db():
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client['test']
    hits_mvideo_collection = db.hits_mvideo
    if DEBUG:
        hits_mvideo_collection.drop()
    return hits_mvideo_collection

def insert_hits(db_hists_mvideo, hits):
    db_hists_mvideo.insert_one(hits)

In [7]:
parse_hits_mvideo()